#Downloading Repos

* Contains code to format visa dataset into mvtect
* Cflow model repo

In [1]:
!git clone https://github.com/amazon-science/spot-diff.git
!git clone https://github.com/gudovskiy/cflow-ad.git

Cloning into 'spot-diff'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 65 (delta 31), reused 47 (delta 17), pack-reused 0
Receiving objects: 100% (65/65), 2.42 MiB | 14.59 MiB/s, done.
Resolving deltas: 100% (31/31), done.
Cloning into 'cflow-ad'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 80 (delta 36), reused 71 (delta 30), pack-reused 2
Receiving objects: 100% (80/80), 818.00 KiB | 5.02 MiB/s, done.
Resolving deltas: 100% (36/36), done.


#Downloading VISA Dataset

In [2]:
!mkdir visa
%cd /content/visa
!wget https://amazon-visual-anomaly.s3.us-west-2.amazonaws.com/VisA_20220922.tar
!tar -xf /content/visa/VisA_20220922.tar
%cd ..

/content/visa
--2023-12-31 08:53:49--  https://amazon-visual-anomaly.s3.us-west-2.amazonaws.com/VisA_20220922.tar
Resolving amazon-visual-anomaly.s3.us-west-2.amazonaws.com (amazon-visual-anomaly.s3.us-west-2.amazonaws.com)... 52.92.251.218, 52.92.185.162, 52.92.149.10, ...
Connecting to amazon-visual-anomaly.s3.us-west-2.amazonaws.com (amazon-visual-anomaly.s3.us-west-2.amazonaws.com)|52.92.251.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1929840640 (1.8G) [application/x-tar]
Saving to: ‘VisA_20220922.tar’

VisA_20220922.tar   100%[===================>]   1.80G  69.6MB/s    in 28s     

2023-12-31 08:54:18 (64.9 MB/s) - ‘VisA_20220922.tar’ saved [1929840640/1929840640]

/content


#Converting VISA dataset to mvtec Format

In [3]:
!mkdir VISA_PCB
%cd /content/spot-diff
!python ./utils/prepare_data.py --split-type 1cls --data-folder /content/visa --save-folder /content/VISA_PCB --split-file ./split_csv/1cls.csv
%cd ..

/content/spot-diff
/content


#Setting up enviornemnt

In [4]:
!pip install FrEIA
!pip install timm

  Preparing metadata (setup.py) ... done
  Created wheel for FrEIA: filename=FrEIA-0.2-py3-none-any.whl size=42759 sha256=2481b856da6369fdf7c62cafc659f3224b6d733e5300a49c657109f3b5495b62
  Stored in directory: /root/.cache/pip/wheels/81/a8/e2/d532a76f72108ac4a340cbe3f86b4f591abfdbd75209a5badb
Successfully built FrEIA
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.0 MB/s eta 0:00:00


#Training Models

* Give path to dataset in main.py
* Change class names in cflow-ad/custom datasets/loader.py
* replace png with jpg in MVTEC dataloader code

* replace  loaing _mask.png with png in ground truth loading part

* Meta epochs are defined in config

* Use c.viz= True in main.py (just in case)

In [ ]:
%cd /content/cflow-ad
!python3 main.py --gpu 0 --pro -inp 512 --dataset mvtec --class-name pcb1 --vis


/content/cflow-ad
LR schedule: [12, 18, 22]
Number of pool layers = 3
CNF coder: 512
CNF coder: 1024
/usr/local/lib/python3.10/dist-packages/FrEIA/modules/all_in_one_block.py:119: UserWarning: Soft permutation will take a very long time to initialize with 1024 feature channels. Consider using hard permutation instead.
  warnings.warn(("Soft permutation will take a very long time to initialize "
CNF coder: 2048
/usr/local/lib/python3.10/dist-packages/FrEIA/modules/all_in_one_block.py:119: UserWarning: Soft permutation will take a very long time to initialize with 2048 feature channels. Consider using hard permutation instead.
  warnings.warn(("Soft permutation will take a very long time to initialize "
/content/VISA_PCB/1cls
pcb1
/content/VISA_PCB/1cls/pcb1/train
/content/VISA_PCB/1cls
pcb1
/content/VISA_PCB/1cls/pcb1/test
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max nu

#Evaluate Trained Model

In [ ]:
!python3 main.py --gpu 0 --pro -enc wide_resnet50_2 --dataset mvtec --action-type norm-test -inp 512 --class-name pcb1 --checkpoint /content/drive/MyDrive/mvtec_wide_resnet50_2_freia-cflow_pl3_cb8_inp512_run0_pcb1_2023-11-26-04:28:49.pt

LR schedule: [12, 18, 22]
Number of pool layers = 3
CNF coder: 512
CNF coder: 1024
/usr/local/lib/python3.10/dist-packages/FrEIA/modules/all_in_one_block.py:119: UserWarning: Soft permutation will take a very long time to initialize with 1024 feature channels. Consider using hard permutation instead.
  warnings.warn(("Soft permutation will take a very long time to initialize "
CNF coder: 2048
/usr/local/lib/python3.10/dist-packages/FrEIA/modules/all_in_one_block.py:119: UserWarning: Soft permutation will take a very long time to initialize with 2048 feature channels. Consider using hard permutation instead.
  warnings.warn(("Soft permutation will take a very long time to initialize "
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker c